<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/cadastro_retencao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importar módulos
import pandas as pd

In [5]:
!pip install pyxlsb

In [27]:
#designar nomes de ficheiros e designar o mês
cadastro_nome="Cadastro_EADW.xlsb"
agentes_nome="AGENTES_TODOS_ENTIDADE_GESTORA_KC13219_29.04.xlsx"

mes="202404"

In [3]:
#fazer upload dos ficheiros
from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()

Saving Cadastro_EADW.xlsb to Cadastro_EADW.xlsb


Saving AGENTES_TODOS_ENTIDADE_GESTORA_KC13219_29.04.xlsx to AGENTES_TODOS_ENTIDADE_GESTORA_KC13219_29.04.xlsx


In [20]:
#importar cadastro
cadastro=pd.read_excel(cadastro_nome, sheet_name=0)
cadastro2=cadastro.copy()
cadastro2.columns=cadastro.iloc[0, :]
cadastro2.columns

Index(['COD MÊS DE FECHO (YYYYMM)', 'COD UTILIZADOR', 'DSC UTILIZADOR',
       'COD RESPONSÁVEL', 'COD ENTIDADE', 'DSC FUNÇÃO', 'DSC CENTRO',
       'DSC PILOTO', 'COD SEGMENTO DE RETENÇÃO', 'QTY CARGA HORÁRIA POR DIA',
       'QTY HORAS TRABALHADAS NO MÊS',
       'QTY HORAS DE ABSENTISMO JUSTIFICADAS NO MÊS',
       'QTY HORAS DE ABSENTISMO INJUSTIFICADAS NO MÊS', 'CAV INBOUND',
       'CAV OUTBOUND'],
      dtype='object', name=0)

In [35]:
#importar ficheiro "agentes todos entidade gestora" do mês

agentes=pd.read_excel(agentes_nome, sheet_name=0)

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [37]:
agentes.columns

Index(['ID Agente', 'Agente', 'ID Entidade do Agente', 'Entidade do Agente',
       'ID Delegação do Agente', 'Delegação do Agente', 'ID Rede do Agente',
       'Rede do Agente', 'Canal de Venda do Agente',
       'Entidade Gestora (Agentes)', 'Segmento de Agente', 'Tipo do Agente',
       'Situação do Agente', 'NIF do Agente'],
      dtype='object')

In [24]:
#definir 1ª linha como cabeçalho

cadastro2.drop(index=0, inplace=True)
print(cadastro2.shape)
cadastro2.head()

(748, 15)


,COD MÊS DE FECHO (YYYYMM),COD UTILIZADOR,DSC UTILIZADOR,COD RESPONSÁVEL,COD ENTIDADE,DSC FUNÇÃO,DSC CENTRO,DSC PILOTO,COD SEGMENTO DE RETENÇÃO,QTY CARGA HORÁRIA POR DIA,QTY HORAS TRABALHADAS NO MÊS,QTY HORAS DE ABSENTISMO JUSTIFICADAS NO MÊS,QTY HORAS DE ABSENTISMO INJUSTIFICADAS NO MÊS,CAV INBOUND,CAV OUTBOUND
1,202404,KJEVANGELISTA,kelly jennifer teixeira evangelista,?,410052,Coordenador,Porto,Proactiva,-1,8,NaN,NaN,NaN,60090418,70090418
2,202404,AFELEITE,andré leite,?,410052,Coordenador,Porto,Proactiva,-1,8,NaN,NaN,NaN,60135317,70135317
3,202404,U80025211,diogo martins oliveira duarte,?,201056,Coordenador,Lisboa,NaN,-1,8,NaN,NaN,NaN,80025211,70025211
4,202404,U80028220,ines alves ferreira lima teixeira,?,201056,Coordenador,Lisboa,NaN,-1,8,NaN,NaN,NaN,80028220,70028220
5,202404,U80010362,marco cesar moreira alves dias,?,203183,Coordenador,Porto,Inbound,-1,8,NaN,NaN,NaN,80010362,70010362


In [34]:
#filtrar dataframe para os dados do mês n

cadastro_mes=cadastro2[cadastro2.iloc[:, 0]==mes]


(387, 15)

In [53]:
#cadastro_mes["CAV INBOUND"]

#dici={"CAV INBOUND": ["Delegação IN_EntidadeGestora", "ID Delegação do Agente"], "CAV INBOUND":["ID Entidade do Agente", "Entidade (EG)"], "CAV INBOUND": ["Tipo do Agente", "Tipo do Agente"]}

for i, j in zip(["Delegação IN_EntidadeGestora", "ID Entidade do Agente", "Tipo do Agente"], ["ID Delegação do Agente", "Entidade (EG)", "Tipo do Agente"]):
  for cav in cadastro_mes.loc[cadastro_mes["CAV INBOUND"]].unique().tolist():
    indice=cadastro_mes.loc[cadastro_mes["CAV INBOUND"]==cav, :].index
    cadastro_mes.loc[indice, i]=agentes.loc[agentes["ID Agente"]==]










Delegação IN_EntidadeGestora
ID Entidade do Agente
Tipo do Agente
